In [1]:
from pathlib import Path

import pandas as pd
import pysam as ps

In [2]:
def coordinate_to_input(coordinate, chr_to_refseq):
    _input = {}
    split = coordinate.split(':')
    _input['chromosome'] = split[0]
    _input['refseq'] = chr_to_refseq[split[0]]
    split = split[1].split('-')
    _input['start'] = int(split[0]) - 1 # fetch is 0-based, half-open
    _input['end'] = int(split[1]) # fetch is 0-based, half-open
    return _input

In [3]:
def get_variants(variant_file, _input):
    return variant_file.fetch(_input['chromosome'], _input['start'], _input['end']) # fetch is 0-based, half-open

In [4]:
def generate_dfs(variant_file, input, filter=None):
    ff = True # first flag
    loci, refs, alts, samples, genotypes = ([] for i in range(5))
    for record in get_variants(variant_file, input):
        loci.append(record.pos)
        refs.append(record.ref)
        alts.append(record.alts[0])
        if ff:
            samples = record.samples.keys()
        gi = 0 # genotypes index
        for i in range(len(record.samples)):
            if (filter is None) or (samples[i] not in filter):
                genotype = record.samples[i]['GT']
                genotypes.append([genotype]) if ff else genotypes[gi].append(genotype)
                gi += 1
        ff = False
    loci_df = pd.DataFrame({'Locus': loci, 'Ref': refs, 'Alt': alts})
    if filter is not None:
        for sample in filter:
            samples.remove(sample)
    samples_df = pd.DataFrame({'Sample': samples, 'Genotype': genotypes})
    return loci_df, samples_df

In [5]:
loi_path = 'Data/C9orf72_Loci.csv' #TODO comma-separated list of loci (of interest) [.csv]
variant_file_path = '1kGP_high_coverage_Illumina.chr9.filtered.SNV_INDEL_SV_phased_panel.vcf.gz' #TODO VCF file [.vcf.gz, etc.] https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/1000G_2504_high_coverage/working/20220422_3202_phased_SNV_INDEL_SV/
#filter_path = 'Data/C9orf72_IDs.csv' #TODO comma-separated list of sample IDs (to be filtered out) [.csv]
n = 3202 #TODO (sample size - |samples filtered out|)

# GRCh38 RefSeq IDs
chr_to_refseq = {'chr1': 'NC_000001.11',
                 'chr2': 'NC_000002.12',
                 'chr3': 'NC_000003.12',
                 'chr4': 'NC_000004.12',
                 'chr5': 'NC_000005.10',
                 'chr6': 'NC_000006.12',
                 'chr7': 'NC_000007.14',
                 'chr8': 'NC_000008.11',
                 'chr9': 'NC_000009.12',
                 'chr10': 'NC_000010.11',
                 'chr11': 'NC_000011.10',
                 'chr12': 'NC_000012.12',
                 'chr13': 'NC_000013.11',
                 'chr14': 'NC_000014.9',
                 'chr15': 'NC_000015.10',
                 'chr16': 'NC_000016.10',
                 'chr17': 'NC_000017.11',
                 'chr18': 'NC_000018.10',
                 'chr19': 'NC_000019.10',
                 'chr20': 'NC_000020.11',
                 'chr21': 'NC_000021.9',
                 'chr22': 'NC_000022.11',
                 'chrX': 'NC_000023.11',
                 'chrY': 'NC_000024.10',
                 'chrMT': 'NC_012920.1'}

# loci of interest
loi = pd.read_csv(loi_path).sort_values('Locus')

# sample IDs to filter (out)
#sample_IDs = set(pd.read_csv(filter_path, header=None)[0]) #TODO used for cohort

input = coordinate_to_input(f'chr9:{loi.iloc[0]["Locus"]}-{loi.iloc[-1]["Locus"]}', chr_to_refseq)
variant_file = ps.VariantFile(variant_file_path)

variants, genotypes = generate_dfs(variant_file, input)
#variants, genotypes = generate_dfs(variant_file, input, sample_IDs) #TODO used for cohort

[W::hts_idx_load3] The index file is older than the data file: /mnt/hgfs/Shared/Data/1000 Genomes/20220422_3202_phased_SNV_INDEL_SV/1kGP_high_coverage_Illumina.chr9.filtered.SNV_INDEL_SV_phased_panel.vcf.gz.tbi


In [6]:
loi

,RefSNP,Locus,ID,nt
0,rs10967976,27544945,H,G
1,rs812858,27545469,G,A
2,rs2453565,27551042,F,C
3,rs12349820,27553878,E,T
4,rs774356,27559723,D,C
5,rs774359,27561051,C,C
6,rs2492816,27565107,B,T
7,rs142843265,27565220,A,A
8,rs2453557,27575465,7,C
9,rs12345062,27578079,6,A


In [7]:
variants

,Locus,Ref,Alt
0,27544945,G,A
1,27545059,A,C
2,27545080,A,G
3,27545119,A,T
4,27545155,C,G
...,...,...,...
1060,27585823,A,G
1061,27585951,A,G
1062,27585964,T,G
1063,27585986,TA,T


In [8]:
genotypes

,Sample,Genotype
0,HG00096,"[(1, 1), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0..."
1,HG00097,"[(0, 1), (0, 0), (0, 1), (0, 0), (0, 0), (0, 0..."
2,HG00099,"[(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0..."
3,HG00100,"[(1, 1), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0..."
4,HG00101,"[(0, 1), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0..."
...,...,...
3197,NA21137,"[(0, 1), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0..."
3198,NA21141,"[(1, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0..."
3199,NA21142,"[(1, 1), (0, 0), (1, 0), (0, 0), (0, 0), (0, 0..."
3200,NA21143,"[(1, 1), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0..."


In [9]:
# multiallelic sanity check
temp = variants.copy()
temp['count'] = 1
temp = temp.groupby('Locus').sum().reset_index()
temp = set(temp[temp['count'] > 1]['Locus'])
temp = variants[variants['Locus'].isin(temp)]
temp[temp['Locus'].isin(set(loi['Locus']))] # empty if no multiallelic sites in the loci of interest

,Locus,Ref,Alt


In [10]:
loci = []
counts = {(1,0): [], (0, 1): [], (1, 1): [], (0, 0): []}
unrecognized = {}
for i, vr in variants[variants['Locus'].isin(loi['Locus'])].iterrows():
    loci.append(vr.Locus)
    count = {(1,0): 0, (0, 1): 0, (1, 1): 0, (0, 0): 0}
    for j, gr in genotypes.iterrows():
        call = gr['Genotype'][i]
        if call in count:
            count[call] += 1
        elif call == (None, None):
            count[(0, 0)] += 1
        else:
            unrecognized[call] = unrecognized.get(call, 0) + 1
    for key in count:
        counts[key].append(count[key])
print(f'Unrecognized call frequencies: {unrecognized}')
result = pd.DataFrame({'Locus': loci, '1/0 Frequency': counts[(1, 0)], '0/1 Frequency': counts[(0, 1)], '1/1 Frequency': counts[(1, 1)], '0/0 Frequency': counts[(0, 0)]})
result['Total Frequency'] = result[['1/0 Frequency', '0/1 Frequency', '1/1 Frequency', '0/0 Frequency']].sum(axis=1)
result = result.merge(variants, how='left', on='Locus')
result = result.merge(loi, how='left', on='Locus')[['RefSNP', 'Locus', 'ID', 'nt', 'Ref', 'Alt', '1/0 Frequency', '0/1 Frequency', '1/1 Frequency', '0/0 Frequency', 'Total Frequency']]
result = result.assign(**{'Heterozygous Percentage': lambda x: ((x['1/0 Frequency'] + x['0/1 Frequency']) / n) * 100, 'Homozygous Ref Percentage': lambda x: (x['0/0 Frequency'] / n) * 100, 'Homozygous Alt Percentage': lambda x: (x['1/1 Frequency'] / n) * 100})
result['Heterozygous nt Percentage'] = result.apply(lambda x: x['Heterozygous Percentage'] if (x['nt'] == x['Ref']) or (x['nt'] == x['Alt']) else 0, axis=1)
result['Homozygous nt Percentage'] = result.apply(lambda x: x['Homozygous Ref Percentage'] if (x['nt'] == x['Ref']) else (x['Homozygous Alt Percentage'] if (x['nt'] == x['Alt']) else 0), axis=1)

result

Unrecognized call frequencies: {}


,RefSNP,Locus,ID,nt,Ref,Alt,1/0 Frequency,0/1 Frequency,1/1 Frequency,0/0 Frequency,Total Frequency,Heterozygous Percentage,Homozygous Ref Percentage,Homozygous Alt Percentage,Heterozygous nt Percentage,Homozygous nt Percentage
0,rs10967976,27544945,H,G,G,A,646,617,694,1245,3202,39.444097,38.881949,21.673954,39.444097,38.881949
1,rs812858,27545469,G,A,C,T,390,441,95,2276,3202,25.952530,71.080575,2.966896,0.000000,0.000000
2,rs2453565,27551042,F,C,C,T,398,439,99,2266,3202,26.139913,70.768270,3.091818,26.139913,70.768270
3,rs12349820,27553878,E,T,T,C,433,404,103,2262,3202,26.139913,70.643348,3.216740,26.139913,70.643348
4,rs774356,27559723,D,C,T,C,454,504,135,2109,3202,29.918801,65.865084,4.216115,29.918801,4.216115
5,rs774359,27561051,C,C,T,C,455,505,136,2106,3202,29.981262,65.771393,4.247345,29.981262,4.247345
6,rs2492816,27565107,B,T,G,A,741,764,526,1171,3202,47.001874,36.570893,16.427233,0.000000,0.000000
7,rs142843265,27565220,A,A,AAG,A,399,442,106,2255,3202,26.264834,70.424735,3.310431,26.264834,3.310431
8,rs2453557,27575465,7,C,T,C,758,786,882,776,3202,48.219863,24.234853,27.545284,48.219863,27.545284
9,rs12345062,27578079,6,A,G,A,696,728,441,1337,3202,44.472205,41.755153,13.772642,44.472205,13.772642


In [11]:
Path('Output').mkdir(parents=True, exist_ok=True)
result.to_csv('Output/C9orf72_1000_GENOMES_nucleotide_frequency.csv', index=False)